# Introduction to atomman: LAMMPS NEB replica dumping

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

The 'neb_replica' dump style is specifically used for setting up NEB calculations in LAMMPS.  LAMMPS sets up NEB calculations by first defining the system for the first replica as is normally done in LAMMPS for other calculations.  The initial configurations in the other replicas are then defined by providing positions for the subset of atoms in the system that are to be subjected to NEB forces during the calculation.  These replica files only contain atom ids and positions for the NEB atoms.  Depending on the LAMMPS options, either a single final replica can be defined and intermediate ones are generated based on linear displacements, or atomic positions can be defined for all replicas with a separate replica file for each.   

__Note:__ The generated files only contain atom ids and positions for a subset of atoms.  As such, this style does not provide a complete description of a system and should only be used for setting up LAMMPS NEB calculations 

*Added version 1.4.11*

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.5.0
Notebook executed on 2024-12-11


Generate test system information (CsCl)

In [2]:
# Generate box
alat = uc.set_in_units(3.2, 'angstrom')
box = am.Box(a=alat, b=alat, c=alat)

# Generate atoms with atype, pos, and a_id properties
atype = [1, 2]
pos = [[0,0,0], [0.5, 0.5, 0.5]]
a_id = [499, 500]
atoms = am.Atoms(pos=pos, atype=atype, a_id=a_id)

# Build system from box and atoms, and scale atoms
system = am.System(atoms=atoms, box=box, scale=True, symbols=['Cs', 'Cl'])

# Print system information
print(system)
system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cs', 'Cl')
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'a_id']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],a_id
0,1,0.0,0.0,0.0,499
1,2,1.6,1.6,1.6,500


## 2. System.dump('neb_replica')<a id='section2'></a>

Generates a LAMMPS NEB replica file based on the System.  __Note__: this format only captures per-atom data for each atom's id and position.  Therefore, it does not offer a lossless representation of a System object.

Parameters

- __f__ (*str or file-like object, option*) File path or file-like object to write the content to.  If not given, then the content is returned as a str.
- __id_key__ (*str, optional*) The name of the atoms property of system to use for the atomic ids. If not given, then the indices of the atoms will be used which should only be done if the atoms in the given system have the same number and order as the reference system used for the initial/first replica.  This atoms property should map the moved atoms to the corresponding ids of the initial/first replica. 
- __id_start0__ (*bool, optional*) LAMMPS ids start at 1 whereas atomman uses atom indices which start at 0.  If idstart0 is True (default) then this indicates that the id_key values are relative to the atomman atoms indices and should be increased by 1 when dumped.  If False, then the id_key values are used as is and assumed to be relative to the LAMMPS atom ids.
- __float_format__ (*str, optional*) c-style formatting string for floating point values.  Default value is '%.13f'.replica   

Returns

- __neb_replica_str__ (*str*) The configuration of the neb replica, returned if f is not given.

Usage notes: The dump style will include all atoms in the System, but typically you only want a subset of atoms from the original reference system.  You can achieve this by defining a new system with only the NEB-subjected atoms in it or by using atoms_ix to slice the system.

The default settings for this dump style will set the atom ids to be equal to their atom index+1.  This should only be used if the atoms to be subjected to NEB forces correspond to the first atoms in the original system, or the replicas contain all atoms of the original system (i.e. all atoms are to be subjected to NEB forces).

In [3]:
replica = system.dump('neb_replica')
print(replica)

2
1 0.0000000000000 0.0000000000000 0.0000000000000
2 1.6000000000000 1.6000000000000 1.6000000000000



For most cases, you will want to set an id_key property to associate with the atom ids.  The id_start0 parameter then controls if id_key values start at 0 (like is used for the atomman atomic indices) or at 1 (like is used for the LAMMPS atom ids). 

With id_start0=True (default) all ids are equal to a_id+1.

In [4]:
# 
replica = system.dump('neb_replica', id_key='a_id')
print(replica)

2
500 0.0000000000000 0.0000000000000 0.0000000000000
501 1.6000000000000 1.6000000000000 1.6000000000000



In [5]:
# With id_start0=False all ids are equal to a_id
replica = system.dump('neb_replica', id_key='a_id', id_start0=False)
print(replica)

2
499 0.0000000000000 0.0000000000000 0.0000000000000
500 1.6000000000000 1.6000000000000 1.6000000000000

